# Pipeline to Train and Test Networks

### Paths & Library Imports

In [2]:
from os.path import isfile, exists
from paths import RETOUCH_PATH, IMAGES_PATH

## Segmentation

### Train-Test Split

Create Train-Test Split for the Fluid Segmentation Task

In [ ]:
from init.folds_split import k_fold_split_segmentation

k = 5

if not (isfile(path="splits\segmentation_train_splits.csv") or isfile(path="splits\segmentation_test_splits.csv")):
    k_fold_split_segmentation(k=k, folders_path=RETOUCH_PATH)
else:
    print("Split already exists. To create a new one please delete the old files.")


### Images Reading and Saving

OCT Volumes Reading and Saving for Segmentation Task

In [ ]:
from init.read_oct import save_segmentation_oct_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\int32")) and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\uint8"))):
    save_segmentation_oct_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Images have already been extracted. To extract them again, please delete the folder with the images.")

# ETA: 2m29s

Topcon_TRAIN070: Volume 70/70: 100%|██████████| 128/128 [00:01<00:00, 91.43img/s] 

All slices have been extracted.
EOF.


OCT Masks Reading and Saving for Segmentation Task

In [ ]:
from init.read_oct import save_segmentation_mask_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\int8")) and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\uint8"))):
    save_segmentation_mask_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Masks have already been extracted. To extract them again, please delete the folder with the images.")

# ETA: 3m4.4s

Topcon_TRAIN070: Volume 70/70: 100%|██████████| 128/128 [00:02<00:00, 47.37img/s]

All masks have been extracted.
EOF.


ROI Masks Extraction for Segmentation Task

In [ ]:
from init.patch_extraction import extractROIMasks

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\int32") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\uint8"))):
    print("First, the images must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\int8") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\uint8"))):
    print("First, the masks must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi")):
    extractROIMasks(oct_path=RETOUCH_PATH , folder_path=IMAGES_PATH, threshold=1e-2)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted ROI masks.")

# ETA: 8h46m01s

ROI_Extraction:   0%|          | 0/70 [00:00<?, ?vol/s]d:\DavidTerroso\D\init\patchExtraction.py:115: UserWarning: Possible precision loss converting image of type float32 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  createROIMask(OCT_slice, OCT_slice_mask, threshold, save_location=save_name, save_location_to_view=save_name_to_view)
ROI_Extraction: 119vol [8:46:01, 265.22s/vol]                        

All ROI masks have been extracted.
EOF.


### Patch Extraction and Saving

Patches Extraction for 2D Segmentation Task

In [ ]:
from init.patch_extraction import extractPatches

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\int8") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\uint8"))):
    print("First, the ROI masks must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\patches\\2D")):
    extractPatches(IMAGES_PATH, patch_shape=(256,128), n_pos=12, n_neg=2, pos=1, neg=0)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted patches.")

Patches Extraction for 2.5D Segmentation Task

In [ ]:
from init.patch_extraction import extractPatches25D

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\int8") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\uint8"))):
    print("First, the ROI masks must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\patches\\2.5D")):
    extractPatches25D(IMAGES_PATH, patch_shape=(256,128), n_pos=12, n_neg=2, pos=1, neg=0)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted patches.")

### Models Tuning

U-Net: Learning Rate

- Learning Rate: 2e-5
- Optimizer: Adam

In [ ]:
from train import train_model

train_model(
    run_name="Run1",
    model_name="UNet",
    device_name="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    val_percent=0.2,
    amp=True,
    patience=100
)

**Results**: the first run. Due to a logic error, the best model was not saved and early stopping was not possible. However, it is interesting to understand the behavior of the U-Net in this conditions. While training loss slowly goes down to 0.26, the validation loss never crosses below the 0.39. Perhaps, a learning rate scheduler would be useful to implement in this conditions, to check if lower validation score is possible. Similarly, implementing this experiment with a lower learning rate may be useful to find a possible lower minimum error, with the expense of requiring a larger number of epochs in training. Also, it is important to mention that in this experiment, the data was obtained assycronously, unlike what was initially proposed, that recommended extracting random patches in every epoch. This later implementation must be compared against the assyncronous, to evaluate the necessaty of the patch extraction every epoch. It is important to note that the patch extraction is very time consuming, with the training process in an epoch taking as long as 15 minutes (without it, it takes 2 minutes).

![Training Error in Run1](./imgs/Run1_training_error.png)

IRF U-Net: Learning Rate

- Learning Rate: 2e-5
- Optimizer: Adam

In [ ]:
from train import train_model

train_model(
    run_name="Run2",
    model_name="UNet3",
    device_name="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=2,
    number_of_channels=1,
    fold_test=1,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    val_percent=0.2,
    amp=True,
    patience=10,
    fluid = "IRF"
)

## Generation

### Train-Test Split

Create Train-Test Split for the Intermediate Slice Generation Task

In [ ]:
from init.folds_split import k_fold_split_generation

k = 5

if not (isfile(path="splits/generation_train_splits.csv") or isfile(path="splits/generation_test_splits.csv")):
    k_fold_split_generation(k=k, folders_path=RETOUCH_PATH)
else:
    print("Split already exists. To create a new one please delete the old files.")

### Images Saving

OCT Volumes Reading and Saving for Generation Task

Note: To make the generation task independent from the segmentation task, the images used for segmentation are being saved again in a different folder. For memory concerns, please adjust the code to reuse those saved previously.

In [ ]:
from init.read_oct import save_generation_oct_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\generation\\int32")) and (exists(IMAGES_PATH + "\\OCT_images\\generation\\uint8"))):
    save_generation_oct_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Images have already been extracted. To extract them again, please delete the folder with the images.")